# Iranian Face Dataset Gathering (Sima Dataset)
##### Creator: Reza Tanakizadeh
##### If you need this dataset send me an email
##### Email: reza.tz780210@gmail.com

In [ ]:
# --- install and loading requirements
!pip3 install instaloader
!pip3 install deepface
!pip3 install urllib3

In [ ]:
# --- import python packages
from deepface import DeepFace
import instaloader
import os
import cv2
import requests
import urllib.request
import matplotlib.pyplot as plt


Directory  /root /.deepface created
Directory  /root /.deepface/weights created


## List of instagram Ids

In [ ]:
# --- select people that we need
ids = ['nedaghasemiofficial',
       'ali_nasirian_']

## Download all image and videos

1. Downlaod imgs & videos
2. clean data from video and unused files

In [ ]:
# --- Get instance
L = instaloader.Instaloader()

# --- Optionally, login or load session
L.login("face.sima", "@Rezatz1378")  
for id in ids:
  # --- Download all posts                         
  posts = instaloader.Profile.from_username(L.context, id).get_posts()
  for idx,post in enumerate(posts):
    if (idx <= 198):
      print(str(idx+1)+":"+str(posts.count)+" -> ", end="")
      L.download_post(post, target=id)
    else:
      break






In [ ]:
for id in ids:
  # --- Delete unused files
  directory = "./"+id
  files_in_directory = os.listdir(directory)
  filtered_files = [file for file in files_in_directory if file.endswith(".xz") or file.endswith(".txt") or file.endswith(".mp4") or file.endswith(".json.xz")]
  for file in filtered_files:
    path_to_file = os.path.join(directory, file)
    os.remove(path_to_file)

  print("Unused files being removed from "+id)

Unused files being removed from raisi_org
Unused files being removed from ali_nasirian_


## Dectect and crop all faces in images that match with that person

Note: you need to upload an image for each person and set it`s filename to instagram_id.jpg

In [ ]:
# -------------------------- UTILS CELL -------------------------------

trained_face_data = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
# --- define Functions
def face_detect(file_name):
  # Choose an image to detect faces in
  img = cv2.imread(file_name)

  # Must convert to greyscale
  grayscaled_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Detect Faces 
  face_coordinates = trained_face_data.detectMultiScale(grayscaled_img)

  img_crop = []

  # Draw rectangles around the faces
  for (x, y, w, h) in face_coordinates:
      img_crop.append(img[y-20:y + h+20, x-20:x + w+20])
  # return image crop
  return img_crop

In [ ]:
for id in ids:
  i=0
  print("-----------"+id + "Started -----------")
  directory = "./"+id
  # --- make result dirs
  os.mkdir(directory+"-cropped")
  files_in_directory = os.listdir(directory)

  # --- get all images in folder
  for file_name in files_in_directory:
    print("File name: ", file_name)
    # --- detect faces
    faces_detected = face_detect(directory+"/"+file_name)
    print("Number of Faces: ",len(faces_detected))
    # --- compare each face with orginal face
    for face in faces_detected:
      try:
        # --- verify that this two face belong to on person
        res = DeepFace.verify(face, id+".jpg", model_name = "ArcFace")
        print("OK: ", str(res["distance"]))
        # --- check Threshold
        if (res["distance"] < 0.73):
          cv2.imwrite(directory+"-cropped/"+str(i)+".jpg", face)
          i = i+1
          print(i)
          print("---- Face Confirmed ----")
        else:
          continue
      except Exception:
        print("Except Error!!")
        continue







Streaming output truncated to the last 5000 lines.
Except Error!!
Except Error!!
Except Error!!
Except Error!!
Except Error!!
File name:  2021-09-25_12-28-08_UTC_4.jpg
Number of Faces:  5
OK:  1.0097203022844126
Except Error!!
Except Error!!
Except Error!!
Except Error!!
File name:  2021-11-04_12-37-11_UTC.jpg
Number of Faces:  2
OK:  0.9754088001437997
Except Error!!
File name:  2021-09-20_12-00-32_UTC_5.jpg
Number of Faces:  7
OK:  0.6214013138585215
35
---- Face Confirmed ----
Except Error!!
OK:  1.022599772525237
OK:  0.8332020466304202
OK:  0.9105711382076687
OK:  1.0446299566066402
OK:  0.9616882263235907
File name:  2021-09-24_17-20-20_UTC_4.jpg
Number of Faces:  6
OK:  1.0089676425935226
OK:  0.8755556426682619
OK:  0.7611074071713689
Except Error!!
OK:  0.8352635155741318
Except Error!!
File name:  2021-12-01_14-28-42_UTC_1.jpg
Number of Faces:  3
Except Error!!
Except Error!!
Except Error!!
File name:  2021-10-01_23-13-21_UTC_8.jpg
Number of Faces:  4
Except Error!!
OK:  0.94

In [ ]:
# --- Find number of images in each dirs
for id in ids:
  directory = "./"+id+"-cropped"
  files_in_directory = os.listdir(directory)
  print("Number of total images in "+id+": "+ str(len(files_in_directory)))

Number of total images in raisi_org: 183
Number of total images in ali_nasirian_: 13


## make Zip file for Downloading

In [ ]:
# --- make ZIP file
from zipfile import ZipFile
from os.path import basename
# create a ZipFile object
for id in ids:
  with ZipFile( id+'.zip', 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(id+"-cropped"):
        for filename in filenames:
            #create complete filepath of file in directory
            filePath = os.path.join(folderName, filename)
            # Add file to zip
            zipObj.write(filePath, basename(filePath))

